## Function to modify the distance between plates

In [9]:
import os
import re

def modify_plates_distance(new_gap):
    # Open the geometry.geo file to read the lines
    with open("geometry.geo", "r") as f:
        lines = f.readlines()
        
    new_lines = []
    for line in lines:

        if 'Point(1) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2)
            # Recreate the line
            new_line = f"Point(1) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)

        elif 'Rotate' in line:
            # Extract the part after 'Rotate {{0, 0, 1},'
            rest = line.split("Rotate {{0, 0, 1},", 1)[1]

            # Find end of coordinate block
            coord_end = rest.find("}")
            coords_str = rest[:coord_end + 1].strip()  # includes closing brace

            # Get the angle part (right after coords)
            angle_str = rest[coord_end + 2:].split("}")[0].strip()

            # Parse and update y value (coords[1])
            coords = coords_str.strip("{}").split(',')
            coords = [c.strip() for c in coords]
            coords[1] = str(new_gap / 2)

            # Reconstruct the line
            new_line = f"Rotate {{ {{0, 0, 1}}, {{{', '.join(coords)}}}, {angle_str} }} {{\n"
            new_lines.append(new_line)
            
        elif 'Point(2) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2)
            # Recreate the line
            new_line = f"Point(2) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(3) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2 + 4)
            # Recreate the line
            new_line = f"Point(3) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(4) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(new_gap / 2 + 4)
            # Recreate the line
            new_line = f"Point(4) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(5) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2)
            # Recreate the line
            new_line = f"Point(5) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(6) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2)
            # Recreate the line
            new_line = f"Point(6) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(7) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2 - 4)
            # Recreate the line
            new_line = f"Point(7) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(8) =' in line:
            # Split the line by commas, remove curly braces, and strip whitespace
            parts = line.split('{')[1].split('}')[0].split(',')
            # Update the second value
            parts[1] = str(-new_gap / 2 - 4)
            # Recreate the line
            new_line = f"Point(8) = {{{','.join(parts)}}};\n"
            new_lines.append(new_line)
        else:
            # If the line doesn't match any of the points, keep it unchanged
            new_lines.append(line)
    
    # Define the directory and file name for saving the new geometry
    directory = "meshes/translation"
    file_name = "geometry_distance_" + str(new_gap) + ".geo"
    
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Define the full file path
    file_path = os.path.join(directory, file_name)
    
    # Write the modified lines to the new geometry file
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with a gap of {new_gap}. Saved to {file_path}")

modify_plates_distance(6)


Geometry updated with a gap of 6. Saved to meshes/translation/geometry_distance_6.geo


# Function to modify overetch of plates

In [10]:
import re
import os


import os
import re

def modify_plates_overetch(overetch):
    # Open the geometry.geo file to read the lines
    with open("geometry.geo", "r") as f:
        lines = f.readlines()
        
    new_lines = []
    for line in lines:

        # first rectangle
        if 'Point(1) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            new_line = f"Point(1) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Rotate' in line:
            # Extract the portion after 'Rotate {{0, 0, 1},'
            rest = line.split("Rotate {{0, 0, 1},", 1)[1]

            # Find the end of the coordinate block (first closing brace after '{x, y, z}')
            coord_end = rest.find("}")
            coords_str = rest[:coord_end+1].strip()  # includes closing brace

            # Extract angle (everything after the coords block and comma)
            angle_str = rest[coord_end+2:].split("}")[0].strip()  # safely handles trailing spaces

            # Parse coordinates and update x (coords[0])
            coords = coords_str.strip("{}").split(',')
            coords = [c.strip() for c in coords]
            coords[0] = str(float(coords[0]) + overetch)

            # Reconstruct the line
            new_line = f"Rotate {{ {{0, 0, 1}}, {{{', '.join(coords)}}}, {angle_str} }} {{\n"
            new_lines.append(new_line)
        elif 'Point(2) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            new_line = f"Point(2) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(3) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            parts[1] = f"{float(parts[1].strip()) - 2 * overetch}"
            new_line = f"Point(3) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(4) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            parts[1] = f"{float(parts[1].strip()) - 2 * overetch}"
            new_line = f"Point(4) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)

        # second rectangle
        elif 'Point(5) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            new_line = f"Point(5) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(6) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            new_line = f"Point(6) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(7) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) - overetch}"
            parts[1] = f"{float(parts[1].strip()) + 2 * overetch}"
            new_line = f"Point(7) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)
        elif 'Point(8) =' in line:
            parts = line.split('{')[1].split('}')[0].split(',')
            parts[0] = f"{float(parts[0].strip()) + overetch}"
            parts[1] = f"{float(parts[1].strip()) + 2 * overetch}"
            new_line = f"Point(8) = {{{', '.join(part.strip() for part in parts)}}};\n"
            new_lines.append(new_line)

        # ignore other lines
        else:
            new_lines.append(line)

    
    # Define the directory and file name for saving the new geometry
    directory = "meshes/overetch"
    file_name = "geometry_overetch_" + str(overetch) + ".geo"
    
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Define the full file path
    file_path = os.path.join(directory, file_name)
    
    # Write the modified lines to the new geometry file
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with an overetch of {overetch}. Saved to {file_path}")

modify_plates_overetch(1)


Geometry updated with an overetch of 1. Saved to meshes/overetch/geometry_overetch_1.geo


## Function to rotate a rectangle

In [11]:
import re
import os
from math import pi

def rotate_upper_plate(new_angle):
    with open("geometry.geo", "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if 'Rotate' in line:
            # Extract text after 'Rotate {{0, 0, 1},'
            rest = line.split("Rotate {{0, 0, 1},", 1)[1]
            
            # Split at the last '}' to isolate the coordinates
            coord_end = rest.rfind('}')
            coords_str = rest[:coord_end+1].strip()  # includes closing }

            # Rebuild the line with the new angle
            new_line = f"Rotate {{ {{0, 0, 1}}, {{{coords_str.strip()}}}, {new_angle} }} {{\n"
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    # Define the directory and file name
    directory = "meshes/rotation"
    file_name = "geometry_angle_" + str(new_angle) + ".geo"
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    # Create the file in the directory
    file_path = os.path.join(directory, file_name)
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Geometry updated with an angle of {new_angle}. Saved to {file_path}")

rotate_upper_plate(0.05)

Geometry updated with an angle of 0.05. Saved to meshes/rotation/geometry_angle_0.05.geo


## Generation of different geometries

In [5]:
import numpy as np
def generate_meshes():
    # Generate 40 equispaced values
    distances = np.linspace(1.5, 2.5, 40)
    overetches = np.linspace(0.1, 0.6, 40)
    angles = np.linspace(1, -1, 40)

    for d in distances:
        modify_plates_distance(d)

    for o in overetches:
        modify_plates_overetch(o)

    for a in angles:
        rotate_upper_plate(a)

generate_meshes()
print("Mesh files generated successfully.")

Mesh files generated successfully.


## Mesh generation

In [ ]:

import gmsh
import os

def generate_mesh_from_geo(geo_path):

    # Initialize Gmsh
    gmsh.initialize()

    # Optional: Hide Gmsh messages
    gmsh.option.setNumber("General.Terminal", 1)

    # Load the .geo file
    gmsh.open(geo_path)

    # Generate 2D or 3D mesh depending on your .geo setup
    gmsh.model.mesh.generate(2)

    # Write the mesh to a .msh file
    msh_path = os.path.splitext(geo_path)[0] + ".msh"
    gmsh.write(msh_path)

    # Finalize Gmsh
    gmsh.finalize()


Info    : Reading 'geometry.geo'...
Info    : Meshing 1D...ence - Adding holes                                                                                           
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Circle)
Info    : [ 80%] Meshing curve 10 (Circle)
Info    : [ 90%] Meshing curve 11 (Circle)
Info    : [100%] Meshing curve 12 (Circle)
Info    : Done meshing 1D (Wall 0.712364s, CPU 0.676786s)
Info    : Meshing 2D...
Info    : Meshing surface 3 (BSpline surface, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 4.99352s, CPU 4.81174s)
Info    : 64208 nodes 128429 elements
Info    : Done reading 'geometry.geo'
Info    : Meshing 2D...
Info    : Meshing surface 3 (BSpline 